Importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib # used to compare the user entered name with the names in the dataset
from sklearn.feature_extraction.text import TfidfVectorizer # used to convert the text data into numerical data(feature vectors)
from sklearn.metrics.pairwise import cosine_similarity # used to calculate the similarity between the feature vectors

Data Collection and Preprocessing

In [2]:
# Load the dataset into a pandas dataframe
movies_data = pd.read_csv('dataset/movies.csv')

In [4]:
# printing the first 5 rows of the dataset
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
# Number of rows and columns in the dataset
movies_data.shape

(4803, 24)

In [103]:
# Selecting the features that are important for the recommendation system
features = ['genres','keywords','original_language','overview','tagline','cast','director']
print(features)

['genres', 'keywords', 'original_language', 'overview', 'tagline', 'cast', 'director']


In [104]:
# Replacing the missing values or the null values with null string(empty string)
for feature in features:
    movies_data[feature] = movies_data[feature].fillna('')

In [105]:
# Separate textual and numeric features
combined_data = (
    movies_data['genres'].astype(str) + ' ' +
    movies_data['keywords'].astype(str) + ' ' +
    movies_data['original_language'].astype(str) + ' ' +
    movies_data['overview'].astype(str) + ' ' +
    movies_data['tagline'].astype(str) + ' ' +
    movies_data['cast'].astype(str) + ' ' +
    movies_data['director'].astype(str)
)

In [106]:
print(combined_data)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  en A newlywed couple's honeymo...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      en When ambitious New York attorney Sam is s...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [107]:
# Convert the textual data into numerical data(feature vectors) using TfidfVectorizer
vectorizer = TfidfVectorizer()

In [108]:
feature_vectors = vectorizer.fit_transform(combined_data)

In [109]:
print(feature_vectors)

  (0, 561)	0.05969696910648952
  (0, 703)	0.06843990680791914
  (0, 9759)	0.08510675234081708
  (0, 23989)	0.07939086944912924
  (0, 10028)	0.07957406226073654
  (0, 6605)	0.14982545342826825
  (0, 5281)	0.1548525564296355
  (0, 10801)	0.11091173652515449
  (0, 25426)	0.24360502997967642
  (0, 29484)	0.08634048724389604
  (0, 5595)	0.17316238876163279
  (0, 25245)	0.13362646954537533
  (0, 8926)	0.026655780261673048
  (0, 13479)	0.03646546344611236
  (0, 27133)	0.08325731422493073
  (0, 239)	0.19709938884067094
  (0, 4770)	0.12497113327809024
  (0, 20111)	0.19253195938485532
  (0, 17026)	0.14319450648940396
  (0, 14028)	0.041676000710361386
  (0, 7831)	0.1797350085371236
  (0, 27420)	0.031004011798625022
  (0, 18254)	0.14764568745192438
  (0, 20046)	0.377340505608285
  (0, 19548)	0.04906580238586927
  :	:
  (4802, 9592)	0.1041404265254575
  (4802, 21394)	0.08974261837592236
  (4802, 9170)	0.07828093766187215
  (4802, 24114)	0.08206838358413666
  (4802, 6139)	0.1077257640744305
  (4802,

Cosine Similarity

In [110]:
# Getting the similarity score(similarity confidence value) using Cosine Similarity
similarity = cosine_similarity(feature_vectors)

In [111]:
print(similarity)

[[1.         0.05148617 0.03392811 ... 0.02801039 0.03099337 0.00770394]
 [0.05148617 1.         0.04417805 ... 0.05125681 0.03149969 0.01564774]
 [0.03392811 0.04417805 1.         ... 0.02694306 0.04797411 0.01414021]
 ...
 [0.02801039 0.05125681 0.02694306 ... 1.         0.03518829 0.03579459]
 [0.03099337 0.03149969 0.04797411 ... 0.03518829 1.         0.03130853]
 [0.00770394 0.01564774 0.01414021 ... 0.03579459 0.03130853 1.        ]]


In [112]:
print(similarity.shape)

(4803, 4803)


In [71]:
# Getting the movie name from the user
movie_name = input('Enter your favourite movie name: ')

In [72]:
# Creating a list with all the movie names in the dataset
list_of_movies = movies_data['title'].tolist()
print(list_of_movies)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [73]:
# Finding the closest match of the user entered movie name with the movie names in the dataset
find_closest_match = difflib.get_close_matches(movie_name, list_of_movies)
print(find_closest_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [74]:
closest_match = find_closest_match[0]
print(closest_match)

Iron Man


In [75]:
movies_data[movies_data['title'] == closest_match]

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
68,68,140000000,Action Science Fiction Adventure,http://www.ironmanmovie.com/,1726,middle east arms dealer malibu marvel comic su...,en,Iron Man,"After being held captive in an Afghan cave, bi...",120.725053,...,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Heroes aren't born. They're built.,Iron Man,7.4,8776,Robert Downey Jr. Terrence Howard Jeff Bridges...,"[{'name': 'Sarah Finn', 'gender': 1, 'departme...",Jon Favreau


In [76]:
# Getting the index of the movie name in the dataset
movie_index = movies_data[movies_data['title'] == closest_match].index[0]
print(movie_index)

68


In [77]:
# Getting a list of similar movies
similarity_scores = list(enumerate(similarity[movie_index]))
print(similarity_scores)

[(0, 0.0492291878118011), (1, 0.035012106647516625), (2, 0.010901244585782857), (3, 0.006700851076677252), (4, 0.02539889638174969), (5, 0.02643885828760479), (6, 0.032498343928373516), (7, 0.1766772094344013), (8, 0.009768145621079308), (9, 0.034582247840803174), (10, 0.040963003378853016), (11, 0.013257365056724918), (12, 0.016453197557663838), (13, 0.010390657024471285), (14, 0.07190061136531782), (15, 0.0267397858458372), (16, 0.12852649155565452), (17, 0.016068444872899882), (18, 0.0213133400818135), (19, 0.04148392775302171), (20, 0.042040192245320226), (21, 0.027693810556213297), (22, 0.009277676935723083), (23, 0.016360544600722494), (24, 0.017514774345325425), (25, 0.0036478172116837107), (26, 0.13487336359511068), (27, 0.03265805537476685), (28, 0.03494077527487931), (29, 0.009750402543197144), (30, 0.08457608364579561), (31, 0.3154952610857998), (32, 0.025210413911540414), (33, 0.073786444108148), (34, 0.005954998971957761), (35, 0.02046578959695626), (36, 0.0240412697365117

In [80]:
# Sorting the movies based on the similarity scores in descending order
sorted_movies = sorted(similarity_scores, key = lambda x:x[1], reverse = True) # here x is similarity_score list, so this will sort the list according to x[1] which is the score value. reverse = True for descending order
print(sorted_movies)

[(68, 1.0), (79, 0.36819609294879757), (31, 0.3154952610857998), (7, 0.1766772094344013), (26, 0.13487336359511068), (16, 0.12852649155565452), (511, 0.11476528635586822), (182, 0.11093273206779782), (64, 0.10550386596235234), (3623, 0.10259981471466437), (94, 0.09674626867689612), (4033, 0.08516511446917314), (30, 0.08457608364579561), (618, 0.08299773824695324), (46, 0.082255005432186), (174, 0.0810917733368321), (783, 0.08094249984923546), (101, 0.07746797449021123), (3133, 0.0774332547278654), (85, 0.07703441743826892), (2875, 0.07588038176395313), (203, 0.0756213711657511), (198, 0.07510339413392544), (3466, 0.07481649473963252), (1740, 0.07479768140808392), (131, 0.07394942059798859), (33, 0.073786444108148), (725, 0.0721889485312123), (1177, 0.0720710542480659), (318, 0.07205991575407633), (2229, 0.07201916363325675), (14, 0.07190061136531782), (1406, 0.07120616784035014), (788, 0.07098920529102579), (1135, 0.07075992157351055), (169, 0.07072532848568515), (138, 0.07034952635835

In [86]:
# Print the top 20 similar movies
print('Top 20 suggested movies are: ')
i = 1
for movie in sorted_movies[1:21]:
    index = movie[0]
    title = movies_data[movies_data.index == index]['title'].values[0]
    print(f"{i}. {title}")
    i += 1

Top 20 suggested movies are: 
1. Iron Man 2
2. Iron Man 3
3. Avengers: Age of Ultron
4. Captain America: Civil War
5. The Avengers
6. X-Men
7. Ant-Man
8. X-Men: Apocalypse
9. Made
10. Guardians of the Galaxy
11. Super
12. Spider-Man 2
13. Mystery Men
14. X-Men: Days of Future Past
15. The Incredible Hulk
16. Mortdecai
17. X-Men: First Class
18. The Devil's Tomb
19. Captain America: The Winter Soldier
20. Two Lovers


Movie Recommendation System

In [114]:
# Getting the movie name from the user
movie_name = input('Enter your favourite movie name: ')

# Creating a list with all the movie names in the dataset
list_of_movies = movies_data['title'].tolist()

# Finding the closest match of the user entered movie name with the movie names in the dataset
find_closest_match = difflib.get_close_matches(movie_name, list_of_movies)
closest_match = find_closest_match[0]

movie_index = movies_data[movies_data['title'] == closest_match].index[0]

similarity_scores = list(enumerate(similarity[movie_index]))

sorted_movies = sorted(similarity_scores, key = lambda x:x[1], reverse = True) # here x is similarity_score list, so this will sort the list according to x[1] which is the score value. reverse = True for descending order

# Print the top 20 similar movies
print('Top 20 suggested movies are: ')
i = 1
for movie in sorted_movies[:20]:
    index = movie[0]
    title = movies_data[movies_data.index == index]['title'].values[0]
    print(f"{i}. {title}")
    i += 1

Top 20 suggested movies are: 
1. RoboCop
2. RoboCop 3
3. Bicentennial Man
4. Street Kings
5. The Fifth Element
6. The Grace Card
7. Listening
8. Bill & Ted's Excellent Adventure
9. American Hero
10. Courageous
11. Terminator 2: Judgment Day
12. The Thief and the Cobbler
13. Stepmom
14. Terminator Genisys
15. Sex Tape
16. Bill & Ted's Bogus Journey
17. Chappie
18. Fugly
19. Robot & Frank
20. Madagascar 3: Europe's Most Wanted
